# Here are the library you need to import

In [1]:
import time
import torch
from torch.utils.data import Dataset
import glob
import scipy.io
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
from natsort import natsorted
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    ToTensord,
    RandAffined,
    RandCropByLabelClassesd,
    SpatialPadd,
    RandAdjustContrastd,
    RandShiftIntensityd,
    ScaleIntensityd,
    NormalizeIntensityd,
    RandScaleIntensityd,
    RandGaussianNoised,
    RandGaussianSmoothd,
    ScaleIntensityRangePercentilesd,
    Resized,
    Transposed,
    RandSpatialCropd,
    RandSpatialCropSamplesd,
    ResizeWithPadOrCropd
)
from monai.transforms import (CastToTyped,
                              Compose, CropForegroundd, EnsureChannelFirstd, LoadImaged,
                              NormalizeIntensity, RandCropByPosNegLabeld,
                              RandFlipd, RandGaussianNoised,
                              RandGaussianSmoothd, RandScaleIntensityd,
                              RandZoomd, SpatialCrop, SpatialPadd, EnsureTyped)
import copy
from cm.karras_diffusion import KarrasDenoiser,karras_sample
from cm.script_util import (
    create_ema_and_scales_fn,
)

from cm.fp16_util import (
    MixedPrecisionTrainer,
    get_param_groups_and_shapes,
    make_master_params,
    master_params_to_model_params,
)

# Build the data loader using the monai library

In [2]:
# Here are the dataloader hyper-parameters, including the batch size,
# image full size, patch size, image spacing, and color channel (usually 1 for medical images)
BATCH_SIZE_TRAIN = 20*1
BATCH_SIZE_TEST = 1
img_full_size = (96,192)
patch_width = 64
patch_size = (patch_width,patch_width)
spacing = (1,1)
patch_num = 2
channels = 1
device = torch.device("mps" if torch.cuda.is_available() else "cpu")

In [3]:
# Here are the dataloader class if your data are .mat files which contains both image and label in a single file. For nii.gz files, see the next block.
# Please see the comment below for your data pre-processing.
# load image-> add channel dimension to the image -> intensity normalization 
# -> padding or crop the boundary to ensure all images have same size -> extract random patches
class CustomDataset(Dataset):
    def __init__(self,imgs_path,labels_path=None, train_flag = True):
        self.imgs_path = imgs_path
        self.labels_path = labels_path
        self.train_flag = train_flag
        files_list = []
        labels_list = []
        for img_path in imgs_path:
            file_list = natsorted(glob.glob(img_path + "*mat"), key=lambda y: y.lower())     
            files_list += file_list
        self.data = []
        self.label = []
        print(f"Found {len(files_list)} files")
        for img_path in files_list:
            class_name = img_path.split("/")[-1]
            self.data.append([img_path, class_name])

        # Now we start to see what data preprocessing we need  
        self.train_transforms = Compose(
                [
                    # Add a new dimension to channel. Must have.
  #                  EnsureChannelFirstd(keys=["image","label"]),
                    
                    # Data normalization: -1 to 1
                     ScaleIntensityd(keys=["image","label"], minv=-1, maxv=1.0),
                    
                    # Pad or crop all images to a uniform size
                    ResizeWithPadOrCropd(
                          keys=["image","label"],
                          spatial_size = img_full_size,
                          constant_values = -1,
                    ),
                    
                    # Randomly extract several patches from the input image
                    RandSpatialCropSamplesd(keys=["image","label"],
                          roi_size = patch_size,
                          num_samples = patch_num,
                          random_size=False,
                          ),
                    ToTensord(keys=["image","label"]),
                ]
            )
        self.test_transforms = Compose(
                [
#                    EnsureChannelFirstd(keys=["image","label"]),
#                     ScaleIntensityd(keys=["image","label"], minv=-1, maxv=1.0),
                    ResizeWithPadOrCropd(
                          keys=["image","label"],
                          spatial_size = img_full_size,
                          constant_values = -1,
                    ),
                    ToTensord(keys=["image","label"]),
                ]
            ) 
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        cao = scipy.io.loadmat(img_path)
        
        # Convert data to numpy arrays and ensure they are 2D
        image_data = cao['image'] if 'image' in cao else cao['data']
        label_data = cao['label'] if 'label' in cao else cao['target']
        
        # Add channel dimension if not present (shape should be [C,H,W])
        if image_data.ndim == 2:
            image_data = image_data[np.newaxis, ...]  # Add channel dimension
        if label_data.ndim == 2:
            label_data = label_data[np.newaxis, ...]  # Add channel dimension
        
        # Create the data dictionary with numpy arrays
        data_dict = {
            'image': image_data.astype(np.float32),  # Convert to float32
            'label': label_data.astype(np.float32)   # Convert to float32
        }
        
        if not self.train_flag:
            affined_data_dict = self.test_transforms(data_dict)   
            img_tensor = affined_data_dict['image']  # Already a tensor from ToTensord
            label_tensor = affined_data_dict['label']  # Already a tensor from ToTensord
        else:
            affined_data_dict = self.train_transforms(data_dict)   
            img = np.zeros([patch_num, patch_width, patch_width])
            label = np.zeros([patch_num, patch_width, patch_width])
            for i, after_l in enumerate(affined_data_dict):
                img[i,:,:] = after_l['image'].squeeze()
                label[i,:,:] = after_l['label'].squeeze()
            img_tensor = torch.from_numpy(img).to(torch.float)
            label_tensor = torch.from_numpy(label).to(torch.float)
        
        return {'image': img_tensor, 'label': label_tensor}

In [4]:
# # Here we use monai to process the nii data. Compare to the mat data, the nii data can have orientation and spacing information.
# # However, compare to .mat data, we must give image and label paths seperately.
# # load image using nibabelreader (read nii) -> add channel dimension to the image -> ensure orientation -> respacing all image to
# # a same spacing -> intensity normalization -> padding or crop the boundary to ensure all images have same size

# class CustomDataset(Dataset):
#     def __init__(self,imgs_path,labels_path, train_flag = True):
#         self.imgs_path = imgs_path
#         self.labels_path = labels_path
#         self.train_flag = train_flag
#         files_list = []
#         labels_list = []
#         for img_path,label_path in zip(imgs_path,labels_path):
#             file_list = natsorted(glob.glob(img_path + "*mat"), key=lambda y: y.lower())     
#             label_list = natsorted(glob.glob(label_path + "*mat"), key=lambda y: y.lower())
#             files_list += file_list
#             labels_list += label_list
#         self.data = []
#         self.label = []
#         for img_path in files_list:
#             class_name = img_path.split("/")[-1]
#             self.data.append([img_path, class_name])
#         for label_path in labels_list:
#             class_name = label_path.split("/")[-1]
#             self.label.append([label_path, class_name])
            
            
#         # Now we start to see what data preprocessing we need  
#         self.train_transforms = Compose(
#                 [
#                     # Read data
#                     LoadImaged(keys=["image","label"],reader='nibabelreader'),
                    
#                     # Add a new dimension to channel. Must have.
#                     AddChanneld(keys=["image","label"]),
                    
#                     # Automatically align the orientation of all patients.
#                     Orientationd(keys=["image"], axcodes="RAS"),
                    
#                     # Automatically align the pixel spacing of all patients.
#                     Spacingd(
#                         keys=["image"],
#                         pixdim=spacing,
#                         mode=("bilinear"),
#                     ),
                    
#                     # Data normalization: -1 to 1
#                     ScaleIntensityd(keys=["image","label"], minv=-1, maxv=1.0),
                    
#                     # Pad or crop all images to a uniform size
#                     ResizeWithPadOrCropd(
#                           keys=["image","label"],
#                           spatial_size = img_full_size,
#                           constant_values = -1,
#                     ),
                    
#                     # Randomly extract several patches from the input image
#                     RandSpatialCropSamplesd(keys=["image","label"],
#                           roi_size = (image_size,image_size),
#                           num_samples = patch_num,
#                           random_size=False,
#                           ),
#                     ToTensord(keys=["image","label"]),
#                 ]
#             )
#         self.test_transforms = Compose(
#                 [
#                     LoadImaged(keys=["image","label"],reader='nibabelreader'),
#                     AddChanneld(keys=["image","label"]),
#                     Orientationd(keys=["image"], axcodes="RAS"),
#                     Spacingd(
#                         keys=["image"],
#                         pixdim=spacing,
#                         mode=("bilinear"),
#                     ),
#                     ScaleIntensityd(keys=["image","label"], minv=-1, maxv=1.0),
#                     ResizeWithPadOrCropd(
#                           keys=["image","label"],
#                           spatial_size = img_full_size,
#                           constant_values = -1,
#                     ),
#                     ToTensord(keys=["image","label"]),
#                 ]
#             ) 
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         img_path, class_name = self.data[idx]
#         label_path, class_name = self.label[idx]
#         data_dict = {"image":img_path,'label':label_path}
#         if not self.train_flag:
#             affined_data_dict = self.test_transforms(data_dict)   
#             img_tensor = affined_data_dict['image'].to(torch.float)
#             label_tensor = affined_data_dict['label'].to(torch.float)   
#         else:
#             affined_data_dict = self.train_transforms(data_dict)   
#             img = np.zeros([patch_num, image_size,image_size])
#             label = np.zeros([patch_num, image_size,image_size])
#             for i,after_l in enumerate(affined_data_dict):
#                 img[i,:,:] = after_l['image'].squeeze()
#                 label[i,:,:] = after_l['label'].squeeze()
#             img_tensor = torch.from_numpy(img).to(torch.float)
#             label_tensor = torch.from_numpy(label).to(torch.float)
        
#         data_tensor = {'image':img_tensor,'label':label_tensor}
#         return data_tensor


# Build the Consistency-Model's network

In [5]:
# Here enter your network parameters:
# num_channels means the initial channels in each block,128 here.
# Length of the channel_mult means the layer#, 4 here.
# channel_mult means the multipliers of the channels (in this case, 128,256,384,512 for the first to the fourth block),
# attention_resulution means we use the transformer blocks in the third to the fourth block
# number of heads, window size in each transformer block
# 
num_channels=128
attention_resolutions="16,8"
channel_mult = (1, 2, 3, 4)
num_heads=[4,4,8,16]
window_size = [[4,4],[4,4],[4,4],[4,4]]
num_res_blocks = [2,2,2,2]
sample_kernel=([2,2],[2,2],[2,2]),

attention_ds = []
for res in attention_resolutions.split(","):
    attention_ds.append(patch_width//int(res))
class_cond = False
use_scale_shift_norm = True

# from Network.Diffusion_model_transformer import *
# Consistency_network = SwinVITModel(
#         image_size=patch_size,
#         in_channels=2,
#         model_channels=num_channels,
#         out_channels=1,
#         dims=2,
#         sample_kernel = sample_kernel,
#         num_res_blocks=num_res_blocks,
#         attention_resolutions=tuple(attention_ds),
#         dropout=0,
#         channel_mult=channel_mult,
#         num_classes=None,
#         use_checkpoint=False,
#         use_fp16=False,
#         num_heads=num_heads,
#         window_size = window_size,
#         num_head_channels=64,
#         num_heads_upsample=-1,
#         use_scale_shift_norm=use_scale_shift_norm,
#         resblock_updown=False,
#         use_new_attention_order=False,
#     ).to(device)

# # Don't forget the target model. You must have this to run the code no matter you use ema or not.
# Consistency_network_target = copy.deepcopy(Consistency_network)

In [6]:
## The Unet we provide here if you want to use it
from Network.Diffusion_model_Unet_2d import *
Consistency_network = UNetModel(
        img_size = patch_size,
        image_size=patch_width,
        in_channels=2,
        model_channels=num_channels,
        out_channels=1,
        dims = 2,
        num_res_blocks=num_res_blocks[0],
        attention_resolutions=tuple(attention_ds),
        dropout=0.,
        sample_kernel=sample_kernel,
        channel_mult=channel_mult,
        num_classes=(NUM_CLASSES if class_cond else None),
        use_checkpoint=False,
        use_fp16=False,
        num_heads=4,
        num_head_channels=64,
        num_heads_upsample=-1,
        use_scale_shift_norm=use_scale_shift_norm,
        resblock_updown=True,
        use_new_attention_order=False,
    ).to(device)

# Don't forget the target model. You must have this to run the code no matter you use ema or not.
Consistency_network_target = copy.deepcopy(Consistency_network)

# Create the consistency-diffusion process

In [7]:
# Setup the hyper-parameters of the consistency model
consistency = KarrasDenoiser(        
        sigma_data=0.5,
        sigma_max=80.0,
        sigma_min=0.002,
        rho=7.0,
        weight_schedule="karras",
        distillation=False,
        loss_norm="l1")

# Call the ema optimizer and ready for start

In [8]:
# Consistency start and end steps in the ema optimization process

start_scales = 2
end_scales = 150
metric = torch.nn.L1Loss()
ema_scale_fn = create_ema_and_scales_fn(
    target_ema_mode='adaptive',
    start_ema=0.95,
    scale_mode='progressive',
    start_scales=start_scales,
    end_scales=end_scales,
    total_steps=800000,
    distill_steps_per_iter=5000,
)

ema_rate  = "0.9999,0.99994,0.9999432189950708"
real_ema_rate = (
            [ema_rate]
            if isinstance(ema_rate, float)
            else [float(x) for x in ema_rate.split(",")]
        )
mp_trainer = MixedPrecisionTrainer(
            model=Consistency_network,
            use_fp16=True,
            fp16_scale_growth=False,
        )
Consistency_model_optimizer = torch.optim.RAdam(mp_trainer.master_params, lr=1e-5,weight_decay = 1e-4)

# Build the training function. Run the training function once = one epoch

In [9]:
# Here we Consistency_network the training process
def train(Consistency_network,Consistency_network_target, Consistency_model_optimizer,data_loader1, consistency,global_step):
    
    #1: set the network and its ema network to training mode
    Consistency_network.train()
    Consistency_network_target.requires_grad_(False)
    Consistency_network_target.train()
    total_samples = len(data_loader1.dataset)
    A_to_B_loss_sum = []
    total_time = 0
    aa = time.time()
    for i, data in enumerate(data_loader1):
        
        #2: Loop the whole dataset, condition (low dose PET, input) and target (high dose PET, target)
        condition = data['image'].view(-1,1,patch_width,patch_width).to(device)
        target = data['label'].view(-1,1,patch_width,patch_width).to(device)
        
        #3: EMA optimizations setup
        A_to_B_param_groups_and_shapes = get_param_groups_and_shapes(
            Consistency_network.named_parameters()
        )
        A_to_B_master_params = make_master_params(A_to_B_param_groups_and_shapes)

        ema_params = [
            copy.deepcopy(A_to_B_master_params)
            for _ in range(len(real_ema_rate))
        ]
        
        ema, num_scales = ema_scale_fn(global_step)
        
                
        #4: Optimize the Consistency_network to correct perform the consistency process
        mp_trainer.zero_grad()
        with torch.cuda.amp.autocast():
            all_loss = consistency.consistency_losses(Consistency_network,
                        target,
                        condition,
                        num_scales,
                        target_model=Consistency_network_target)
            
            A_to_B_loss = all_loss['loss'].mean()
            A_to_B_loss_sum.append(A_to_B_loss.detach().cpu().numpy())
            
        mp_trainer.backward(A_to_B_loss)
        mp_trainer.optimize(Consistency_model_optimizer)

        #5: _update_ema()
        for rate, params in zip(real_ema_rate, ema_params):
            for targ, src in zip(params, A_to_B_master_params):
                targ.detach().mul_(0.99).add_(src, alpha=1 - 0.99)
        
        target_model_param_groups_and_shapes = get_param_groups_and_shapes(
            Consistency_network_target.named_parameters()
        )
        target_model_master_params = make_master_params(
            target_model_param_groups_and_shapes
        )
        
        target_ema, scales = ema_scale_fn(global_step)
        with torch.no_grad():
            for targ, src in zip(target_model_master_params, A_to_B_master_params):
                targ.detach().mul_(target_ema).add_(src, alpha=1 - target_ema)

            master_params_to_model_params(
                target_model_param_groups_and_shapes,
                target_model_master_params,
            )

        total_time += time.time()-aa
        
        
        #6: Print out loss for every 100 iteration, just for monitoning 
        global_step += 1
        if i % 100 == 0:
            print('optimization time: '+ str(time.time()-aa))
            print('[' +  '{:5}'.format(i * BATCH_SIZE_TRAIN) + '/' + '{:5}'.format(total_samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_loader1)) + '%)]  A_to_B_Loss: ' +
                  '{:6.7f}'.format(np.nanmean(A_to_B_loss_sum)))

        
    average_loss = np.nanmean(A_to_B_loss_sum) 
    print('Averaged loss is: '+ str(average_loss))
    return global_step

# Build the testing function.

In [10]:
# Use the window sliding method to denoise the whole PET image. Must used it.
# For example, if your whole image is 96x192, and our window size is 64x64, so the function will automatically sliding down
# the whole image with a certain overlapping ratio

# The window size (img_size) is shown in the "Build the data loader using the monai library" section.
# img_size: the size of sliding window
# img_num: the number of sliding window in each process, only related to your gpu memory, it will still run through the whole volume
# overlap: the overlapping ratio

# Set up the step# for your inference
consistency_num = 3
steps = np.round(np.linspace(1.0, 150.0, num=consistency_num))
def diffusion_sampling(Low_dose,A_to_B_model):
    sampled_images = karras_sample(
                        consistency,
                        A_to_B_model,
                        shape=Low_dose.shape,
                        condition=Low_dose,
                        sampler="multistep",
                        steps = 151,
                        ts = steps,
                        device = device)
    return sampled_images

# Patch-based inference parameter
overlap = 0.75
mode ='constant'
back_ground_intensity = -1
Inference_patch_number_each_time = 40
from monai.inferers import SlidingWindowInferer
inferer = SlidingWindowInferer(patch_size, Inference_patch_number_each_time, overlap=overlap,
                               mode =mode ,cval = back_ground_intensity, sw_device=device,device = device)

def evaluate(Consistency_network,epoch,path,data_loader1,best_loss,global_step):
    Consistency_network.eval()
    prediction = []
    true = []
    img = []
    loss_all = []
    aa = time.time()
    with torch.no_grad():
        for i, data in enumerate(data_loader1):

                Low_dose =  data['image'].to(device)        
                High_dose = data['label'].to(device)
                
                High_dose_samples = inferer(Low_dose,diffusion_sampling,Consistency_network)  
                loss = metric(High_dose_samples,High_dose)
                print('MSE: '+ str(loss))
                img.append(Low_dose.cpu().numpy())
                true.append(High_dose.cpu().numpy())
                prediction.append(High_dose_samples.cpu().numpy())    
                loss_all.append(loss.cpu().numpy())
                
        print('optimization time: '+ str(time.time()-aa))
        print('average loss: '+str(np.mean(loss_all)))
        return np.mean(loss_all)

# Start the training and testing

In [11]:
# Enter your data folder
training_set1 = CustomDataset(['/Users/fnayres/upenn/Full-dose-Whole-body-PET-Synthesis-from-Low-dose-PET-Using-Consistency-Model/data/pet_38_aligned/imagesTr_full_2d/'], train_flag=True)
testing_set1 = CustomDataset(['/Users/fnayres/upenn/Full-dose-Whole-body-PET-Synthesis-from-Low-dose-PET-Using-Consistency-Model/data/pet_38_aligned/imagesTs_full_2d'],train_flag=False)

# Enter your data reader parameters
params = {'batch_size': BATCH_SIZE_TRAIN,
          'shuffle': True,
          'pin_memory': True,
          'drop_last': False}
train_loader1 = torch.utils.data.DataLoader(training_set1, **params)

params = {'batch_size': 320,
          'shuffle': False,
          'pin_memory': True,
          'drop_last': False}
test_loader1 = torch.utils.data.DataLoader(testing_set1, **params)
# Enter your total number of epoch
N_EPOCHS = 500

# Enter the address you save the checkpoint and the evaluation examples
path ="/Users/fnayres/upenn/Full-dose-Whole-body-PET-Synthesis-from-Low-dose-PET-Using-Consistency-Model/result"
Consistency_network_path = path+'A_to_B_ViTRes1.pt' # Use your own path6
best_loss = 1
global_step = 0
if not os.path.exists(Consistency_network_path):
  os.makedirs(Consistency_network_path) 
train_loss_history, test_loss_history = [], []

# Uncomment this when you resume the checkpoint
# Consistency_network.load_state_dict(torch.load(Consistency_network_path),strict=True) 
Consistency_network_model_ema = copy.deepcopy(Consistency_network)
for epoch in range(0, N_EPOCHS):
    print('Epoch:', epoch)
    start_time = time.time() 
    global_step = train(Consistency_network,Consistency_network_model_ema, Consistency_model_optimizer,
                        train_loader1,consistency,global_step)
    print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')
    if epoch % 1 == 0:
        average_loss = evaluate(Consistency_network,epoch,path,test_loader1,best_loss,global_step)
        if average_loss < best_loss:
            print('Save the latest best model')
            torch.save(Consistency_network.state_dict(), Consistency_network_path)
            best_loss = average_loss
print('Execution time')

Found 2 files
Found 0 files
Epoch: 0


/var/folders/0c/9q35mp5n4hz_yvyz_9cj6r4m0000gn/T/ipykernel_96443/1858427631.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/Users/fnayres/miniconda3/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Logging to /var/folders/0c/9q35mp5n4hz_yvyz_9cj6r4m0000gn/T/openai-2025-03-26-09-11-18-806459
optimization time: 4.654666900634766
[    0/    2 (  0%)]  A_to_B_Loss: 3.8468025
Averaged loss is: 3.8468025
Execution time:  4.67 seconds
optimization time: 0.0003018379211425781
average loss: nan
Epoch: 1


/Users/fnayres/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/fnayres/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


optimization time: 4.136078834533691
[    0/    2 (  0%)]  A_to_B_Loss: 1.2402568
Averaged loss is: 1.2402568
Execution time:  4.16 seconds
optimization time: 0.0002651214599609375
average loss: nan
Epoch: 2
optimization time: 4.317837953567505
[    0/    2 (  0%)]  A_to_B_Loss: 2.9402893
Averaged loss is: 2.9402893
Execution time:  4.34 seconds
optimization time: 0.0002799034118652344
average loss: nan
Epoch: 3
optimization time: 4.17822003364563
[    0/    2 (  0%)]  A_to_B_Loss: 1.1516764
Averaged loss is: 1.1516764
Execution time:  4.20 seconds
optimization time: 0.0001659393310546875
average loss: nan
Epoch: 4
optimization time: 3.4818549156188965
[    0/    2 (  0%)]  A_to_B_Loss: 0.3139412
Averaged loss is: 0.31394124
Execution time:  3.51 seconds
optimization time: 0.00017881393432617188
average loss: nan
Epoch: 5
optimization time: 4.315496921539307
[    0/    2 (  0%)]  A_to_B_Loss: 1.2412069
Averaged loss is: 1.2412069
Execution time:  4.34 seconds
optimization time: 0.00016